In [1]:
import pandas  as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import gc
gc.collect()

0

In [2]:
train = pd.read_csv("../input/train.csv", dtype = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }, skiprows = list(range(1,150000000)))

In [3]:
train.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,156608,3,1,10,173,2017-11-09 05:19:45,NaN,0
1,111186,12,1,22,178,2017-11-09 05:19:45,NaN,0
2,143839,12,1,17,178,2017-11-09 05:19:45,NaN,0
3,87609,18,1,19,107,2017-11-09 05:19:45,NaN,0
4,123924,2,1,25,469,2017-11-09 05:19:45,NaN,0


In [4]:
train.is_attributed.value_counts(normalize=True)

0    0.997474
1    0.002526
Name: is_attributed, dtype: float64

In [5]:
train.shape

(34903891, 8)

In [6]:
train['hourofday'] = pd.to_datetime(train["click_time"]).dt.hour

In [7]:
import lightgbm as lgb
from sklearn.metrics import *
from sklearn.model_selection import *

In [8]:
feats = ['ip', 'app', 'device', 'os', 'channel', 'hourofday']
X = train[feats]
y = train.is_attributed

In [9]:
del train

In [10]:
y = y.astype(np.uint8)

In [11]:
X["hourofday"] = X["hourofday"].astype("uint8")

In [24]:
cvlist1 = list(StratifiedKFold(5, random_state=1).split(X, y))
cvlist2 = list(StratifiedShuffleSplit(3, test_size=0.1, random_state=1).split(X, y))
cvlist3 = list(StratifiedShuffleSplit(3, test_size=0.1, random_state=2).split(X, y))
cvlist4 = list(StratifiedShuffleSplit(3, test_size=0.05, random_state=3).split(X, y))

In [32]:
model = lgb.LGBMClassifier(num_leaves=255, n_jobs=-1, n_estimators=500, subsample=0.7, colsample_bytree=0.7,
                           verbose=10, device="gpu", num_bins=31)

In [30]:
from utils import outoffold_crossvalidator, shuffle_crossvalidator

In [33]:
%%time
outoffold_crossvalidator(model, X, y, cvlist=cvlist1)

Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.931276
[20]	valid_0's auc: 0.929621
[30]	valid_0's auc: 0.9376
[40]	valid_0's auc: 0.945643
[50]	valid_0's auc: 0.948361
[60]	valid_0's auc: 0.948255
[70]	valid_0's auc: 0.949129
[80]	valid_0's auc: 0.95
[90]	valid_0's auc: 0.949851
[100]	valid_0's auc: 0.950008
[110]	valid_0's auc: 0.949593
[120]	valid_0's auc: 0.949201
[130]	valid_0's auc: 0.948441
[140]	valid_0's auc: 0.948557
Early stopping, best iteration is:
[99]	valid_0's auc: 0.950244
Score for this fold is  0.9502438775533375
Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.910488
[20]	valid_0's auc: 0.91543
[30]	valid_0's auc: 0.924364
[40]	valid_0's auc: 0.931458
[50]	valid_0's auc: 0.931848
[60]	valid_0's auc: 0.931307
[70]	valid_0's auc: 0.931392
[80]	valid_0's auc: 0.932515
[90]	valid_0's auc: 0.933182
[100]	valid_0's auc: 0.93305
[110]	valid_0's auc: 0.93279
[120]	valid_0's auc: 0.932774
[130]	valid_0's 

(array([2.78518357e-04, 1.37839076e-04, 1.16299775e-04, ...,
        1.63612598e-01, 1.62369801e-01, 1.62494824e-01]),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 [])

In [15]:
%%time
outoffold_crossvalidator(model, X, y, cvlist=cvlist1)

Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.896306
[20]	valid_0's auc: 0.913052
[30]	valid_0's auc: 0.911174
[40]	valid_0's auc: 0.919046
[50]	valid_0's auc: 0.944434
[60]	valid_0's auc: 0.952046
[70]	valid_0's auc: 0.955266
[80]	valid_0's auc: 0.956759
[90]	valid_0's auc: 0.956643
[100]	valid_0's auc: 0.956805
[110]	valid_0's auc: 0.957305
[120]	valid_0's auc: 0.957728
[130]	valid_0's auc: 0.957786
[140]	valid_0's auc: 0.95765
[150]	valid_0's auc: 0.957577
[160]	valid_0's auc: 0.957648
[170]	valid_0's auc: 0.957555
Early stopping, best iteration is:
[124]	valid_0's auc: 0.957864
Score for this fold is  0.9578640504704719
Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.898151
[20]	valid_0's auc: 0.915149
[30]	valid_0's auc: 0.91087
[40]	valid_0's auc: 0.931119
[50]	valid_0's auc: 0.945237
[60]	valid_0's auc: 0.950787
[70]	valid_0's auc: 0.951632
[80]	valid_0's auc: 0.95163
[90]	valid_0's auc: 0.952805
[100]	val

(array([3.15135271e-04, 1.98850257e-04, 1.49126886e-04, ...,
        6.27803027e-04, 4.70937077e-05, 6.11750926e-04]),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 [])

In [16]:
%%time
shuffle_crossvalidator(model, X, y, cvlist=cvlist2)


Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.912498
[20]	valid_0's auc: 0.923586
[30]	valid_0's auc: 0.920692
[40]	valid_0's auc: 0.932885
[50]	valid_0's auc: 0.951149
[60]	valid_0's auc: 0.957935
[70]	valid_0's auc: 0.960812
[80]	valid_0's auc: 0.962522
[90]	valid_0's auc: 0.963111
[100]	valid_0's auc: 0.964293
[110]	valid_0's auc: 0.964707
[120]	valid_0's auc: 0.965026
[130]	valid_0's auc: 0.965172
[140]	valid_0's auc: 0.965439
[150]	valid_0's auc: 0.965488
[160]	valid_0's auc: 0.965769
[170]	valid_0's auc: 0.965886
[180]	valid_0's auc: 0.965819
[190]	valid_0's auc: 0.966059
[200]	valid_0's auc: 0.96599
[210]	valid_0's auc: 0.965815
[220]	valid_0's auc: 0.965359
[230]	valid_0's auc: 0.965186
Early stopping, best iteration is:
[185]	valid_0's auc: 0.966109
Score for this fold is  0.9661093545169657
Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.911795
[20]	valid_0's auc: 0.924858
[30]	valid_0's auc: 0.922706
[

(array([3.85477169e-04, 8.79651385e-05, 8.46851862e-04, ...,
        3.22501242e-03, 1.35301086e-04, 8.70125782e-05]),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 [])

In [22]:
gc.collect()

12

In [17]:
%%time
shuffle_crossvalidator(model, X, y, cvlist=cvlist3)

Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.908707
[20]	valid_0's auc: 0.921661
[30]	valid_0's auc: 0.918983
[40]	valid_0's auc: 0.929263
[50]	valid_0's auc: 0.952492
[60]	valid_0's auc: 0.957141
[70]	valid_0's auc: 0.959453
[80]	valid_0's auc: 0.961588
[90]	valid_0's auc: 0.963215
[100]	valid_0's auc: 0.963839
[110]	valid_0's auc: 0.964578
[120]	valid_0's auc: 0.96468
[130]	valid_0's auc: 0.964894
[140]	valid_0's auc: 0.965232
[150]	valid_0's auc: 0.96543
[160]	valid_0's auc: 0.965472
[170]	valid_0's auc: 0.965618
[180]	valid_0's auc: 0.965731
[190]	valid_0's auc: 0.965642
[200]	valid_0's auc: 0.965664
[210]	valid_0's auc: 0.965417
[220]	valid_0's auc: 0.965377
Early stopping, best iteration is:
[178]	valid_0's auc: 0.965739
Score for this fold is  0.9657390948104712
Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.907206
[20]	valid_0's auc: 0.922004
[30]	valid_0's auc: 0.917741
[40]	valid_0's auc: 0.926686
[50

(array([2.13065300e-05, 1.45808224e-04, 1.59388386e-04, ...,
        1.05504045e-04, 3.15976415e-02, 2.13740374e-03]),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 [])

In [24]:
gc.collect()

90

In [18]:
%%time
shuffle_crossvalidator(model, X, y, cvlist=cvlist4)

Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.911843
[20]	valid_0's auc: 0.926513
[30]	valid_0's auc: 0.919804
[40]	valid_0's auc: 0.928962
[50]	valid_0's auc: 0.954111
[60]	valid_0's auc: 0.960581
[70]	valid_0's auc: 0.962846
[80]	valid_0's auc: 0.963472
[90]	valid_0's auc: 0.964259
[100]	valid_0's auc: 0.965372
[110]	valid_0's auc: 0.966142
[120]	valid_0's auc: 0.96655
[130]	valid_0's auc: 0.966758
[140]	valid_0's auc: 0.966968
[150]	valid_0's auc: 0.967099
[160]	valid_0's auc: 0.966332
[170]	valid_0's auc: 0.966612
[180]	valid_0's auc: 0.966724
[190]	valid_0's auc: 0.966887
[200]	valid_0's auc: 0.96699
[210]	valid_0's auc: 0.967035
[220]	valid_0's auc: 0.96787
[230]	valid_0's auc: 0.96779
[240]	valid_0's auc: 0.967439
[250]	valid_0's auc: 0.967326
[260]	valid_0's auc: 0.967302
Early stopping, best iteration is:
[215]	valid_0's auc: 0.967936
Score for this fold is  0.9679360542269096
Training until validation scores don't improve for 50 rounds.
[

(array([1.12160105e-04, 2.32372735e-04, 1.93416968e-04, ...,
        8.43843116e-05, 3.11282131e-04, 1.06465201e-04]),
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 [])

In [19]:
X.shape

(34903891, 6)